In [7]:
import pickle
from typing import Type, Callable, Optional

import numpy as np
from datasets import load_dataset
from tqdm import tqdm
from datetime import datetime

from reasoners import LanguageModel, Reasoner, SearchAlgorithm
from reasoners.algorithm import MCTS

from world_model import MATHWorldModel
from search_config import MATHConfig
import utils

from datasets import Dataset
import os
import re
import json

In [8]:
def data_reader(dataset,dataset_path, split=None, sample_size=100):
    questions = []
    answers = []
    options = []
    filename = os.path.join(dataset_path, 'AQuA.json')
    with open(filename, 'r') as file:
        lines = file.readlines()
        if split is not None:
            start, end = split
            lines = lines[start:end]
        for line in lines:
            data = json.loads(line)
            if isinstance(data, dict):
                options_list = data['options']
                options_dict = {}
                for option in options_list:
                    match = re.search(r'([A-E])\)[^0-9]*([\d.]+)', option)
                    if match:
                        options_dict[match.group(1)] = float(match.group(2))
                question_with_options = data['question'] + "\n" + "\n".join(data['options'])
                questions.append(question_with_options)
                # answers.append(options_dict.get(data['correct']))
                answers.append(data['correct'])
                options.append(options_list)
            else:
                raise ValueError("Unexpected data format")
    return Dataset.from_dict({"question": questions, "answer": answers, "options":options})

In [9]:
datasetname: str = 'AQuA'
dataset_path: str = '/data/yueshan/llm-reasoners/examples/AQuA/dataset/AQuA'
dataset = data_reader(datasetname, dataset_path)


In [10]:
print(dataset[1]["answer"])

E
